## Optimizing `quorum` parameter and page detector weights in `EnsemblePageDetector`

In [1]:
from itertools import product, chain, combinations

from video699.video.annotated import (
    AnnotatedSampledVideoScreenDetector,
    evaluate_event_detector,
    get_videos,
)
from video699.page.ensemble import EnsemblePageDetector
from video699.page.imagehash import ImageHashPageDetector
from video699.page.vgg16 import KerasVGG16PageDetector
from video699.page.siamese import KerasSiamesePageDetector
from video699.quadrangle.rtree import RTreeDequeConvexQuadrangleTracker
from video699.event.screen import ScreenEventDetector

import numpy as np
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
def accuracy(quorum, imagehash_weight, vgg16_weight, siamese_weight):
    num_successes_total = 0
    num_trials_total = 0
    for annotated_video in get_videos().values():
        convex_quadrangle_tracker = RTreeDequeConvexQuadrangleTracker(2)
        screen_detector = AnnotatedSampledVideoScreenDetector()
        documents = annotated_video.documents.values()
        page_detector = EnsemblePageDetector(
            {
                ImageHashPageDetector(documents): imagehash_weight,
                KerasVGG16PageDetector(documents): vgg16_weight,
                KerasSiamesePageDetector(documents): siamese_weight,
            },
            quorum,
        )
        screen_event_detector = ScreenEventDetector(
            annotated_video,
            convex_quadrangle_tracker,
            screen_detector,
            page_detector
        )
        num_successes, num_trials = evaluate_event_detector(annotated_video, screen_event_detector)
        num_successes_total += num_successes
        num_trials_total += num_trials
    accuracy = 1.0 * num_successes_total / num_trials_total
    return accuracy

In [3]:
def parameters(refinement):
    weight = np.linspace(0.0, 1.0, num=refinement + 1)
    parameters = [
        (quorum, *weights)
        for weights in product(weight, weight, weight) if sum(weights) == 1.0
        for quorum in set(
            float(sum(weight_subset))
            for weight_subset in chain(*[
                combinations(weights, r + 1)
                for r in range(len(weights))
            ])
        ) if quorum > 0.0
    ]
    return tqdm(parameters)

In [4]:
def print_results(accuracies):
    best_parameter, best_accuracy = max(accuracies.items(), key=lambda x: x[1])
    best_quorum, *best_weights = best_parameter
    print('Optimal parameters (accuracy {}):'.format(best_accuracy))
    print('- quorum:  {}'.format(best_quorum))
    print('- weights: {}'.format(best_weights))

In [5]:
accuracies = dict()
for parameter in parameters(refinement=5):
    if parameter not in accuracies:
        accuracies[parameter] = accuracy(*parameter)
print_results(accuracies)

100%|██████████| 69/69 [27:27:46<00:00, 1432.85s/it]   

Optimal parameters (accuracy 0.8557457212713936):
- quorum:  1.0
- weights: [0.0, 0.0, 1.0]
